In [ ]:
#r "nuget: Microsoft.SemanticKernel"
#r "nuget: SkiaSharp"

#!import config/Settings.cs
#!import config/Utils.cs
#!import config/SkiaUtils.cs

using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.TextToImage;
using Microsoft.SemanticKernel.ChatCompletion;
using Microsoft.SemanticKernel.Connectors.OpenAI;

In [ ]:
using Kernel = Microsoft.SemanticKernel.Kernel;

#pragma warning disable SKEXP0001, SKEXP0010

var settings = Settings.LoadFromFile("config/settings.json");


var builder = Kernel.CreateBuilder();
builder.AddAzureOpenAIChatCompletion(settings.model, settings.azureEndpoint,settings.apiKey);
builder.AddAzureOpenAITextToImage("dall-e-3", settings.azureEndpoint, settings.apiKey);

var kernel = builder.Build();


var dallE = kernel.GetRequiredService<ITextToImageService>();
var chatGPT = kernel.GetRequiredService<IChatCompletionService>();

In [ ]:
var systemMessage = 
"""
You're chatting with a user. 
Instead of replying directly to the user provide a description of a cartoonish image that expresses what you want to say.
The user won't see your message, they will see only the image. 
Describe the image with details in one sentence.
""";

var chat = new ChatHistory(systemMessage);

In [ ]:
#pragma warning disable SKEXP0001

Func<string, Task> Chat = async (string input) => {
    Console.WriteLine($"User: {input}");
    chat.AddUserMessage(input);

    var assistantReply = await chatGPT.GetChatMessageContentAsync(chat, new OpenAIPromptExecutionSettings());
    chat.AddAssistantMessage(assistantReply.Content);

    Console.WriteLine($"\nBot:");
    var imageUrl = await dallE.GenerateImageAsync(assistantReply.Content, 1024, 1024);
    Console.WriteLine($"\n{imageUrl}");
    await SkiaUtils.ShowImage(imageUrl, 1024, 1024);
    Console.WriteLine($"\n{assistantReply.Content}");
};

In [ ]:
while (true)
{
    var userMessage = await InteractiveKernel.GetInputAsync("Your message");
    await Chat(userMessage);
}